In [1]:
# 引入需要的包裝
import pandas as pd
import numpy as np
import pyvisa
import time
import math
import matplotlib.pyplot as plt

import sys
import threading
from PyQt6 import QtWidgets, QtCore
from PyQt6.QtCore import QThread, pyqtSignal
from PyQt6.QtGui import QImage, QPixmap
import cv2
from UI import Ui_MainWindow

from MSO64B import MSO_64B  # 匯入 MSO_64B 類
from AFG31101 import AFG_31101  # 匯入 AFG_#1101 類
from RF150A100D import RF_150A100D #匯入 RF_150A100D 類
from HIOKI8450 import HIOKI_8450 #匯入HIOKI_8450 類

### AFG31101

===============設置AFG31101之特定功能=================

* 

In [2]:
def check_output_state(afg, output_state):
    if output_state == 'ON': #判斷輸出狀態，選擇要操作在哪一個輸出狀態DEF
        afg.output_on()
    else:
        afg.output_off()

def check_amp_limit(afg, rf, amplitude_limit, amplitude):  # 檢查振幅是否超過限制
    amp_num = float(amplitude['magnitude'])  # 將當下振幅大小的字串型態轉為數字型態
    if amp_num >= amplitude_limit:  # 判斷振幅是否大於AMPLITUDE_LIMIT
        print(f'Magnitude of amplitude cannot over {amplitude_limit} voltage')
        gain_set = calculate_gain_set(gain=0)  # 計算增益設置值
        self.rf.set_gain(gain, gain_set) #先將功率放大器增益調為零
        self.rf.power_off() #資料儲存完畢，關閉RF
        afg.output_off()  # 將OUTPUT關閉
        print(f'Close signal generator and power amplifier')

### RF150A100D

===============設置RF150A100D之特定功能=================

* check_power_gain:檢查增益之大小是否小於零
* calculate_gain_set:計算丟給儀器之增益參數設定該為多少

In [3]:
def check_power_state(rf, power_state):
    if power_state == 'ON': #判斷書出狀態，選擇def
        rf.power_on()
    else:
        rf.power_off()
        
def check_power_gain(rf, gain):
    if gain < 0:
        rf.power_off()
        print(f"gain you set {gain} can't < 0")

def calculate_gain_set(gain):
    """
    計算並返回設置增益的值
    透過設定的gain去計算給儀器的gain_set該為多少
    """
    ratio = 4095 / 100  # MAX和MIN的比例
    gain_set = math.ceil(ratio * gain)  # 使用 math函數進行無條件進位
    gain_set = min(gain_set, 4095)  # 限制 gain_set 最大值為 4095
    gain_set = str(gain_set).zfill(4)  # 保持四個字元
    return gain_set

def calculate_gain_from_set(gain_set):
    """
    根據儀器返回的四位數 gain_set，計算並返回實際的增益大小，並取整數
    """
    # 刪除前綴 "G" 並轉換為數字
    gain_value = int(gain_set[1:])
    
    # 根據比例 4095 對應 100%，反推出增益
    ratio = 100 / 4095  # MAX和MIN的比例
    gain = gain_value * ratio
    
    # 返回四捨五入到整數的增益
    return round(gain)

### MSO64B

===============設置MSO64B之特定功能=================

* calculate_horizonscale_cursor:計算水平刻度之大小
* set_vertical_scale:設置垂直刻度
* wait_for_population:等待示波器抓取波形
* save_query_values:抓取示波器Delay、Amplitude的大小
* calculate_attenuation:計算通道振幅要調整的倍率大小
* adjust_attenuation:調整通道的振幅倍率

In [4]:
def wait_for_population(mso, threshold=20):
    while True: #重複查詢採樣次數(population)
        pop_num = mso.query_population()
        print(f"Current population: {pop_num}") #列出每0.5秒的pop次數
        if pop_num >= threshold: #當pop次數大於等於100，結束while迴圈
            print(f"Poplution採樣數量已超過{threshold}")
            break #跳脫迴圈
        time.sleep(0.5)  # 防止過於頻繁的查詢，調整每0.5秒查詢一次

def set_cursor_range(afg,mso):
    """
    設置示波器的cursor range
    [因為如果一開始示波器的horizon scale沒有調好，抓到的freq值會過大，調整錯誤，因此直接從AFG訊號產生器抓值]
    """
    freq = float(afg.query_frequency()) #從afg抓頻率值
    print(f"the frequency of excitaton is {freq} Hz")
    
    cursor_A_position = -2 * (1 / freq)  # 設cursor_A為負1.5波形週期sec
    cursor_B_position = 2 * (1 / freq)  # 設cursor_B為正1.5波形週期sec
    mso.set_cursor(cursor_A_position, cursor_B_position)
    
def set_horizon_scale(afg, mso):
    """
    設置示波器的horizon scale
    [因為如果一開始示波器的horizon scale沒有調好，抓到的freq值會過大，調整錯誤，因此直接從AFG訊號產生器抓值]
    """
    freq = float(afg.query_frequency()) #從afg抓頻率值
    print(f"the frequency of signal is {freq}Hz")
    
    horizon_scale = (1 / freq) * (4 / 10)
    mso.set_horizon_scale(horizon_scale) #自動調整horizon scale
    return horizon_scale
    
def set_vertical_scale(mso):
    clip_channels = ['CH1','CH2','CH3']#分別測試ch1、ch2、ch3是否clipping?

    channel_meas = {'CH1': 'MEAS3','CH2': 'MEAS4','CH3': 'MEAS5'} #每個channel對應的amplitude測量項
    
    for clip_channel in clip_channels:
        while True: #重複測試到最後沒通道clipping & vertical scale都設定到80%
            all_channels_clipped = False

            time.sleep(0.25)
            clip_result = float(mso.detect_clip(clip_channel)) 
            #print(f"clipping state:{clip_result}") #顯示1代表有clipping，0代表沒有clipping
            if clip_result == 1: #clipping時
                clip_vertical_scale = float(mso.query_vertical_scale(clip_channel))
                new_clip_vertical_scale = clip_vertical_scale + 40e-3 #將當下的vertical scale以20mV/div為step加上去
                print(f"{clip_channel} is clipping")
                mso.set_vertical_scale(clip_channel, new_clip_vertical_scale) #設定新的vertical scale
                time.sleep(0.05) #最後調整到沒有clipping等待久一點，讓下面程序執行正常
            elif clip_result == 0: # 當channel沒有clipping時
                time.sleep(0.2)
                meas = channel_meas[clip_channel] #當下的clip_channel對應的meas是什麼?
                time.sleep(1)
                print(f"{clip_channel} no clipping")
                clip_amplitude = float(mso.query_mean_value(meas)) # 看該通道的amplitude大小
                print(f"amplitude of {clip_channel} is {clip_amplitude}")
                new_vertical_scale = clip_amplitude / 6.5  #將amplitude大小除以7即為理想的60~80% vertical scale
                mso.set_vertical_scale(clip_channel, new_vertical_scale) # 設定新的vertical scale
                time.sleep(0.25)
                clip_result = float(mso.detect_clip(clip_channel)) 
                if clip_result == 1:
                    time.sleep(0.05)
                    print(f'{clip_channel} is clipping')
                    print('readjust the vertical scale')
                    all_channels_clipped = True  # 代表還有channel clipping
                else:
                    print(f'vertical state of {clip_channel} is good')
                    break

def set_vref_deskew(afg, mso, deskew_vref):
    freq = float(afg.query_frequency()) #查詢測試環境所設定之激勵源的頻率
    deskew_ch1 = (deskew_vref)/(360 * freq) #計算Deskew角度要調整的秒數
    mso.set_CH1_deskew(deskew_ch1)

### Core

===============計算Core=================

* 

In [5]:
def calculate_core_parameters(core_He, core_OD, core_ID):
    """
    計算鐵芯的有效路徑長度(l_e)、有效截面積(A_e)和有效體積(V_e)
    """
    # 計算 C1 和 C2
    h = core_He
    d1 = core_OD
    d2 = core_ID

    C1 = (2 * math.pi) / (h * math.log(d1 / d2))
    C2 = (4 * math.pi * (1/d2 - 1/d1)) / (h**2 * math.log(d1 / d2)**3)

    # 計算 le, Ae, Ve
    le = (C1**2) / C2
    Ae = C1 / C2
    Ve = le * Ae
    # print(f"有效路徑長度 (le): {le} mm")
    # print(f"有效截面積 (Ae): {Ae} mm^2")
    # print(f"有效體積 (Ve): {Ve} mm^3")

    return le, Ae, Ve

### Save values

===============抓取各種值=================

* 

In [6]:
def calculate_deltaB_values(mso, sequence, core_effective_area, turn):
    """
    用來儲存向儀器(示波器)查詢之值的包裝
    """
    B_max = mso.query_mean_value('MEAS9')
    B_min = mso.query_mean_value('MEAS10')
    calculate_delta_B = (B_max - B_min) / (turn * core_effective_area * 0.001)
    delta_B_mT = calculate_delta_B * 1000000
    
    time.sleep(0.05)
    print(f"The maximum B value is {B_max} T")
    print(f"The minimum B value is {B_min} T")
    print(f"The delta of B is {delta_B_mT} T")
    
    return B_max, B_min, calculate_delta_B, delta_B_mT

In [7]:
def calculate_final_power_loss(sequence, Pcore_true_power, PL_true_power, Pcore_true_power_deskew, PL_true_power_deskew, core_effective_volume):
    """
    套公式計算 k_factor, Pcore 和 Pcore_cv
    """
    k_factor = (PL_true_power_deskew['2'][sequence-1][1] - PL_true_power['2'][sequence-1][1]) / (Pcore_true_power_deskew['1'][sequence-1][1] - Pcore_true_power['1'][sequence-1][1])
    print(f"k factor:{k_factor}")
    Pcore = (Pcore_true_power['1'][sequence-1][1] - (k_factor * PL_true_power['2'][sequence-1][1]))
    print(f"Core Real power loss:{Pcore}")
    Pcore_cv = (Pcore / core_effective_volume) * 1000000
    print(f"Every volume of Core Real power loss:{Pcore_cv}")

    return k_factor, Pcore, Pcore_cv

### Measurement result

===============抓取各種值=================

* 

In [8]:
def save_mso_power_values(mso, sequence):
    """
    用來儲存向儀器(示波器)查詢之值的包裝
    """
    power1_value = float(mso.query_true_power('1'))
    power2_value = float(mso.query_true_power('2'))
    
    time.sleep(0.05)
    print(f"The true power of 1 is {power1_value} W")
    print(f"The true power of 2 is {power2_value} W")
    
    return power1_value, power2_value

In [9]:
def acquire_measurement(mso, i, core_effective_area, turn, max_B, min_B, delta_B, Pcore_true_power, PL_true_power, deskew=False):
    B_max, B_min, calculate_delta_B, delta_B_mT = calculate_deltaB_values(mso, i, core_effective_area, turn)
        
    # 更新 max_B 和 min_B 字典
    if deskew:
        if 'MEAS9' not in max_B:
            max_B['MEAS9'] = []
        max_B['MEAS9'].append((i, B_max))

        if 'MEAS10' not in min_B:
            min_B['MEAS10'] = []
        min_B['MEAS10'].append((i, B_min))
        
        delta_B[i] = delta_B_mT
    else:
        if 'MEAS9' not in max_B:
            max_B['MEAS9'] = []
        max_B['MEAS9'].append((i, B_max))
        # print("max_B:", max_B)

        if 'MEAS10' not in min_B:
            min_B['MEAS10'] = []
        min_B['MEAS10'].append((i, B_min))
        # print("min_B:", min_B)
        
        delta_B[i] = delta_B_mT
        # print("delta_B:", delta_B)

    # 量測損耗功率
    power1_value, power2_value = save_mso_power_values(mso, i)

    if deskew:
        if '1' not in Pcore_true_power:
            Pcore_true_power['1'] = []
        Pcore_true_power['1'].append((i, power1_value))
        # print("Pcore true power_deskew:", Pcore_true_power)
        
        if '2' not in PL_true_power:
            PL_true_power['2'] = []
        PL_true_power['2'].append((i, power2_value))
        # print("PL true power_deskew:", PL_true_power)
    else:
        if '1' not in Pcore_true_power:
            Pcore_true_power['1'] = []
        Pcore_true_power['1'].append((i, power1_value))
        #print("Pcore true power:", Pcore_true_power)
        
        if '2' not in PL_true_power:
            PL_true_power['2'] = []
        PL_true_power['2'].append((i, power2_value))
        #print("PL true power:", PL_true_power)
    # return delta_B_mT

### adjust_rf_afg

===============自動調整&判斷激勵源大小=================

* 

In [10]:
def adjust_rf_afg(mso, rf, afg, i, core_effective_area, turn, target_deltaB, gain, amplitude, amplitude_limit):
    """
    調整 RF 增益以達到目標 delta B 值
    減少
    """
    tolerance_1 = 1
    tolerance_2 = 3
    tolerance_3 = 5
    tolerance_4 = 8
    tolerance_5 = 15 
    tolerance_6 = 25

    time.sleep(0.5)
    # 查詢初始的 delta B 值
    B_max, B_min, calculate_delta_B, delta_B_mT = calculate_deltaB_values(mso, i, core_effective_area, turn)
    
    while abs(target_deltaB - delta_B_mT) > tolerance_1:
        query_gain = rf.query_gain()  # 查詢當前增益
        current_gain = calculate_gain_from_set(query_gain)  # 反推並取整的增益
        print(f"Gain now:{current_gain}")
        if current_gain <= 45:
            print("Gain大於45")
            step_afg_1 = 2
            step_afg_2 = 4
            step_afg_3 = 6
            step_afg_4 = 9
            decrease_afg_1 = 3
            decrease_afg_2 = 5
            
            step_rf_1 = 1
            step_rf_2 = 2
            step_rf_3 = 3
            step_rf_4 = 5
            step_rf_5 = 7
            step_rf_6 = 9
            decrease_rf_1 = 1
            decrease_rf_2 = 3
            decrease_rf_3 = 7
        else:
            print("Gain小於45")
            step_afg_1 = 0
            step_afg_2 = 1
            step_afg_3 = 1
            step_afg_4 = 2
            decrease_afg_1 = 5
            decrease_afg_2 = 8
            
            step_rf_1 = 1
            step_rf_2 = 1
            step_rf_3 = 1
            step_rf_4 = 1
            step_rf_5 = 1
            step_rf_6 = 2
            decrease_rf_1 = 1
            decrease_rf_2 = 2
            decrease_rf_3 = 5
            
        tolerance = target_deltaB - delta_B_mT
        print(f"實際 delta B: {delta_B_mT}mT 與預期 delta B: {target_deltaB}mT 差了 {tolerance}mT")

        if tolerance < -tolerance_5:
            print(f"實際deltaB大於預期deltaB {tolerance_3}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) - decrease_afg_2  # 移除 'E-3' 並減少
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已減少 {decrease_afg_2}V")
            gain -= decrease_rf_3
            print(f"增益減少 {decrease_rf_3}")
        elif tolerance < -tolerance_4:
            print(f"實際deltaB大於預期deltaB {tolerance_1}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) - decrease_afg_2  # 移除 'E-3' 並減少
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已減少 {decrease_afg_2}V")
            gain -= decrease_rf_2
            print(f"增益減少 {decrease_rf_1}")
        elif tolerance < -tolerance_1:
            print(f"實際deltaB大於預期deltaB {tolerance_1}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) - decrease_afg_1  # 移除 'E-3' 並減少
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已減少 {decrease_afg_1}V")
            gain -= decrease_rf_1
            print(f"增益減少 {decrease_rf_1}")
        elif tolerance > tolerance_6:
            print(f"delta B 誤差大於 {tolerance_6}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) + step_afg_4  # 移除 'E-3' 並增加 3mV
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已增加 {step_afg_4}V")
            gain += step_rf_6
            print(f"增益增加 {step_rf_6}")
        elif tolerance > tolerance_5:
            print(f"delta B 誤差介於 {tolerance_5} ~ {tolerance_6}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) + step_afg_3  # 移除 'E-3' 並增加 2mV
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已增加 {step_afg_3}V")
            gain += step_rf_5
            print(f"增益增加 {step_rf_5}")
        elif tolerance > tolerance_4:
            print(f"delta B 誤差介於 {tolerance_4} ~ {tolerance_5}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) + step_afg_2  # 移除 'E-3' 並增加 2mV
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已增加 {step_afg_2}V")
            gain += step_rf_4
            print(f"增益增加 {step_rf_4}")
        elif tolerance > tolerance_3:
            print(f"delta B 誤差介於 {tolerance_3} ~ {tolerance_4}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) + step_afg_2  # 移除 'E-3' 並增加 2mV
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已增加 {step_afg_2}V")
            gain += step_rf_3
            print(f"增益增加 {step_rf_3}")
        elif tolerance > tolerance_2:
            print(f"delta B 誤差介於 {tolerance_2} ~ {tolerance_3}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) + step_afg_1  # 移除 'E-3' 並增加 2mV
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已增加 {step_afg_1}V")
            gain += step_rf_2
            print(f"增益增加 {step_rf_2}")
        elif tolerance > tolerance_1:
            print(f"delta B 誤差介於 {tolerance_1} ~ {tolerance_2}")
            amplitude_value = float(amplitude['magnitude'].replace('E-3', '')) + step_afg_1  # 移除 'E-3' 並增加 2mV
            amplitude['magnitude'] = f"{amplitude_value}E-3" #重新設定amplitude
            print(f"AFG 振幅已增加 {step_afg_1}V")
            gain += step_rf_1
            print(f"增益增加 {step_rf_1}")
        else:
             print(f"delta B已達預期的{target_deltaB}")
            
        check_amp_limit(afg, rf, amplitude_limit, amplitude)  # 檢查振幅是否超過限制
        afg.set_amplitude(amplitude['magnitude'], amplitude['unit'])

        check_power_gain(rf, gain)
        gain_set = calculate_gain_set(gain)
        rf.set_gain(gain, gain_set)

        set_vertical_scale(mso)

        time.sleep(2)
        B_max, B_min, calculate_delta_B, delta_B_mT = calculate_deltaB_values(mso, i, core_effective_area, turn)
        
    return amplitude, gain

In [11]:
def format_time(seconds):
    """將記錄下來測量時間的秒數轉為分:秒"""
    minutes = int(seconds // 60)
    remaining_seconds = int(seconds % 60)
    return f"{minutes}:{remaining_seconds}"

In [12]:
def every_measurement_time(i, start_time, measurement_start_time, measurement_times, accumulated_times):
    """計算每次測量的結束時間及其花費時間"""
    measurement_end_time = time.time()
    measurement_duration = measurement_end_time - measurement_start_time
    measurement_times.append(measurement_duration)

    accumulated_time = measurement_end_time - start_time
    accumulated_times.append(accumulated_time)

    formatted_measurement_duration = format_time(measurement_duration)
    formatted_accumulated_time = format_time(accumulated_time)

    print(f"[第{i}次測量]花費時間: {formatted_measurement_duration}")
    print(f"程式執行累計時間: {formatted_accumulated_time}")

    return measurement_duration

### 主程式

===============開始control=================

* 

In [13]:
def generate_expect_deltaB(start_deltaB, end_deltaB, step_of_deltaB):
    # 當只有一個測量點時，直接返回 [start_deltaB]
    if start_deltaB == end_deltaB and step_of_deltaB == 0:
        return [start_deltaB]
    
    # 當有多個測量點時
    expect_deltaB = []
    
    # 初始化當前測量點為 start_deltaB
    current_point = start_deltaB
    
    # 確保測量點保持整數
    start_deltaB = int(start_deltaB)
    end_deltaB = int(end_deltaB)
    step_of_deltaB = int(step_of_deltaB)

    # 生成從 start 到 end 並且每次增加 step 的測量點
    while current_point < end_deltaB:
        expect_deltaB.append(current_point)
        current_point += step_of_deltaB
    
    # 確保最後一個測量點是 end_deltaB
    if current_point >= end_deltaB:
        expect_deltaB.append(end_deltaB)

    return expect_deltaB

In [14]:
# 檢查參數的函式
def check_parameters(core_type, core_OD, core_ID, core_He, turn, frequency, start_deltaB, end_deltaB, step_of_deltaB, check_parameters, amplitude_state):
    """
    驗證輸入的鐵芯和測試條件參數是否正確
    """
    errors = []
    if amplitude_state == '0201' or amplitude_state == '0301':
        errors.append("請將功率放大器調製'REMOTE'控制狀態。")
    
    # 檢查 core_type 是否填寫
    if not core_type:
        errors.append("請填寫鐵芯的 Material (材質)。")
    if not check_parameters:
        errors.append("請填寫csv檔檔名。")
        
    if core_OD == 0 or core_ID == 0 or core_He == 0:
        errors.append("請填寫鐵芯參數，外徑、內徑和高度。")

    # 檢查 core_OD > core_ID
    if core_OD <= core_ID:
        errors.append("鐵芯參數外徑(OD)與內徑(ID)有誤。")

    if turn == 0:
        errors.append("請填寫繞組圈數。")

    # 檢查 frequency 範圍
    if not (500 <= frequency <= 3000):
        errors.append("頻率必須在 500 到 3000 之間。")

    # 檢查 start_deltaB 的範圍和大小
    if start_deltaB > 250:
        errors.append("鐵芯之最大值磁通密度不可大於250 mT。")
    if start_deltaB == 0:
        errors.append("請填寫Start DeltaB。")
    if end_deltaB == 0:
        errors.append("請填寫End DeltaB。")
    if start_deltaB > end_deltaB:
        errors.append("Delta B之開始大小不可大於結束大小。")
    if start_deltaB != end_deltaB:
        if step_of_deltaB == 0:
            errors.append("請填寫開始測量點與結束測量點之間的step大小。")
    
    return errors

In [15]:
class AutoTestWorker(QThread):
    result_ready = pyqtSignal(int, float, float, list, float)
    test_complete = pyqtSignal(dict)

    def __init__(self, sequences, expect_deltaB, mso, rf, afg, hio, core_effective_area, core_effective_volume, turn, gain, amplitude, amplitude_limit, deskew_vref, start_time, measurement_times, accumulated_times):
        super().__init__()
        self.sequences = sequences
        self.expect_deltaB = expect_deltaB
        self.mso = mso
        self.rf = rf
        self.afg = afg
        self.hio = hio
        self.core_effective_area = core_effective_area
        self.core_effective_volume = core_effective_volume
        self.turn = turn
        self.gain = gain
        self.amplitude = amplitude
        self.amplitude_limit = amplitude_limit
        self.deskew_vref = deskew_vref
        self.start_time = start_time
        self.measurement_times = measurement_times
        self.accumulated_times = accumulated_times
        self._is_running = True  # 初始化運行狀態為 True

        # 將這個值的字典變量定義為實例變量
        self.vin_amplitude = {}
        self.power_gain = {}
        self.max_B = {}
        self.min_B = {}
        self.delta_B = {}
        self.Pcore_true_power = {}
        self.PL_true_power = {}
        self.max_B_deskew = {}
        self.min_B_deskew = {}
        self.delta_B_deskew = {}
        self.Pcore_true_power_deskew = {}
        self.PL_true_power_deskew = {}
        self.k_factor = {}
        self.Pcore = {}
        self.Pcore_cv = {}
        self.temperature = {}
        self.measurement_spend_time = {}

    def run(self):
        # 你可以直接使用這些實例變量，而不需要在這裡再次定義字典變量
        """
        Autoset horizon & vertical scale
        """
        self.mso.clear()
        wait_for_population(self.mso, threshold=2)  # 先等待pop到一定數量再抓數值
        set_horizon_scale(self.afg, self.mso)
    
        self.mso.clear()
        wait_for_population(self.mso, threshold=2)
        set_vertical_scale(self.mso)

        """
        測量過程的邏輯在這裡執行
        """
        for i in self.sequences:
            if not self._is_running:
                break  # 如果 _is_running 為 False，跳出循環並停止測試
                
            print(f"[第{i}次測量]")

            # 計算每次測量開始的時間
            measurement_start_time = time.time()

            # 自動調整激勵源
            target_deltaB = self.expect_deltaB[i - 1]
            wait_for_population(self.mso, threshold=5)
            amplitude, gain = adjust_rf_afg(self.mso, self.rf, self.afg, i, self.core_effective_area, self.turn, target_deltaB, self.gain, self.amplitude, self.amplitude_limit)
            
            self.gain = gain
            self.amplitude = amplitude
            
            self.vin_amplitude[i] = amplitude['magnitude']
            self.power_gain[i] = gain

            # 開始抓值
            self.mso.set_CH1_deskew_zero()
            wait_for_population(self.mso)

            acquire_measurement(self.mso, i, self.core_effective_area, self.turn, self.max_B, self.min_B, self.delta_B, self.Pcore_true_power, self.PL_true_power)

            # 鐵芯溫度查詢
            data_result = self.hio.query_data()
            self.temperature[i] = [round(float(temp.strip().replace('+', '')), 3) for temp in data_result.split(',') if temp.strip()]

            print("對Vref Deskew")
            set_vref_deskew(self.afg, self.mso, self.deskew_vref)

            self.mso.clear()
            wait_for_population(self.mso)

            acquire_measurement(self.mso, i, self.core_effective_area, self.turn, self.max_B_deskew, self.min_B_deskew, self.delta_B_deskew, self.Pcore_true_power_deskew, self.PL_true_power_deskew, deskew=True)

            # 計算 Pcore_cv
            self.k_factor[i], self.Pcore[i], self.Pcore_cv[i] = calculate_final_power_loss(i, self.Pcore_true_power, self.PL_true_power, self.Pcore_true_power_deskew, self.PL_true_power_deskew, self.core_effective_volume)

            # 計算每次測量的結束時間及其花費時間
            self.measurement_spend_time[i] = every_measurement_time(i, self.start_time, measurement_start_time, self.measurement_times, self.accumulated_times)

            # 發送測量結果給主界面
            self.result_ready.emit(i, self.delta_B[i], self.Pcore_cv[i], self.temperature[i], self.measurement_spend_time[i])

        # 測量完畢，關閉儀器
        self.end_test_procedure()

        # 測試完成後，回傳所有測試結果
        final_data = {
            "sequences": self.sequences,
            "vin_amplitude": self.vin_amplitude,
            "power_gain": self.power_gain,
            "delta_B": self.delta_B,
            "Pcore_true_power": self.Pcore_true_power,
            "PL_true_power": self.PL_true_power,
            "delta_B_deskew": self.delta_B_deskew,
            "Pcore_true_power_deskew": self.Pcore_true_power_deskew,
            "PL_true_power_deskew": self.PL_true_power_deskew,
            "k_factor": self.k_factor,
            "Pcore": self.Pcore,
            "Pcore_cv": self.Pcore_cv,
            "temperature": self.temperature,
            "measurement_times": self.measurement_times,
            "accumulated_times": self.accumulated_times
        }
        self.test_complete.emit(final_data)  # 發送測試完成信號及結果

    def stop(self):
        """設置停止標誌，終止測試"""
        self._is_running = False

    def end_test_procedure(self):
        """測試結束時關閉設備"""
        print("測量完畢，正在關閉設備")
        gain = 0
        gain_set = calculate_gain_set(gain)  # 計算增益設置值
        self.rf.set_gain(gain, gain_set)  # 將功率放大器增益調為零
        time.sleep(0.5)
        self.afg.output_off()  # 關閉AFG
        self.rf.power_off()  # 關閉RF

In [16]:
class MainWindow(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        self.instrument = None  # 儀器實例變量
        self.setup_control()

        # 初始設置鐵芯參數
        # self.initialize_core_parameters()

        #設定振幅、頻率範圍
        self.ui.test_frequency_value.setMinimum(0)   # 設置最小值
        self.ui.test_frequency_value.setMaximum(10000)  # 設置最大值 (將上限設置為 1000)
        self.ui.test_start_deltaB_value.setMinimum(0)   # 設置最小值
        self.ui.test_start_deltaB_value.setMaximum(300)  # 設置最大值 (將上限設置為 1000)
        self.ui.test_end_deltaB_value.setMinimum(0)   # 設置最小值
        self.ui.test_end_deltaB_value.setMaximum(300)  # 設置最大值 (將上限設置為 1000)
        self.ui.test_deltaB_step_value.setMinimum(0)   # 設置最小值
        self.ui.test_deltaB_step_value.setMaximum(100)  # 設置最大值 (將上限設置為 1000)
        
    def setup_control(self):
        """
        Picture display
        """
        self.ECIE_img_path = 'ECIE.jpg'
        self.display_ECIE_img()
        self.circuit_img_path = 'Circuit.jpg'
        self.display_circuit_img()
        self.core_img_path = 'Core.jpg'
        self.display_core_img()
        """
        Instruments connection action
        """
        self.ui.address_refresh.clicked.connect(self.refresh_instruments)
        self.ui.connect_button.clicked.connect(self.connect_instruments)
        self.ui.close_connect_button.clicked.connect(self.close_connect_instruments)
        """
        Start test
        """
        self.ui.add_measurement.clicked.connect(self.add_test_measurement)
        self.ui.test_button.clicked.connect(self.start_test)
        self.ui.stop_test.clicked.connect(self.stop_test)
        """
        core calculate
        """
        self.ui.calculate_core.clicked.connect(self.calculate_core)

    """
    Picture display
    """
    def display_ECIE_img(self):
        try:
            # 確認圖像路徑是否正確
            self.img = cv2.imread(self.ECIE_img_path)
            
            # 獲取圖像的尺寸
            height, width, channel = self.img.shape  # 獲取圖片的高度、寬度、通道數
            bytesPerLine = 3 * width  # 計算每行字節數
            
            # 創建QImage 並檢查轉換是否正確
            self.qimg = QImage(self.img, width, height, bytesPerLine, QImage.Format.Format_RGB888).rgbSwapped()
            
            # 設置圖片到 QLagel
            self.ui.ECIE_image.setPixmap(QPixmap.fromImage(self.qimg))  # 將 QImage 設置到 QLabel
            #self.update_status("Circuit Image displayed successfully.")
            
        except Exception as e:
            self.update_status(f"Error displaying image: {str(e)}")
            
    def display_circuit_img(self):
        try:
            # 確認圖像路徑是否正確
            self.img = cv2.imread(self.circuit_img_path)
            
            # 獲取圖像的尺寸
            height, width, channel = self.img.shape  # 獲取圖片的高度、寬度、通道數
            bytesPerLine = 3 * width  # 計算每行字節數
            
            # 創建QImage 並檢查轉換是否正確
            self.qimg = QImage(self.img, width, height, bytesPerLine, QImage.Format.Format_RGB888).rgbSwapped()
            
            # 設置圖片到 QLagel
            self.ui.circuit_image.setPixmap(QPixmap.fromImage(self.qimg))  # 將 QImage 設置到 QLabel
            #self.update_status("Circuit Image displayed successfully.")
            
        except Exception as e:
            self.update_status(f"Error displaying image: {str(e)}")

    def initialize_core_parameters(self):
        """
        初始化鐵芯參數，並做必要的輸入防呆處理
        """
        try:
            # 嘗試抓取 UI 上的輸入值，並做空值和格式檢查
            core_type = self.ui.core_type.text() or "DefaultType"
            core_OD = float(self.ui.core_od.text() or 0)  # 單位:mm
            core_ID = float(self.ui.core_id.text() or 0)  # 單位:mm
            core_He = float(self.ui.core_he.text() or 0)  # 單位:mm
            turn = float(self.ui.core_turn.text() or 0)   # 單位:turn
        except ValueError:
            # 如果輸入無效，顯示錯誤訊息
            QtWidgets.QMessageBox.warning(self, "輸入錯誤", "請確保所有鐵芯參數的輸入為數值格式。")

    def display_core_img(self):
        try:
            # 確認圖像路徑是否正確
            self.img = cv2.imread(self.core_img_path)

            # 獲取圖像的尺寸
            height, width, channel = self.img.shape  # 獲取圖片的高度、寬度、通道數
            bytesPerLine = 3 * width  # 計算每行字節數
            
            # 創建QImage 並檢查轉換是否正確
            self.qimg = QImage(self.img, width, height, bytesPerLine, QImage.Format.Format_RGB888).rgbSwapped()
            
            # 設置圖片到 QLagel
            self.ui.core_image.setPixmap(QPixmap.fromImage(self.qimg))  # 將 QImage 設置到 QLabel
            self.update_status("Core Image displayed successfully.")
            
        except Exception as e:
            self.update_status(f"Error displaying image: {str(e)}")
    """
    Instruments connection
    """
    def refresh_instruments(self):
        """刷新可用的儀器地址並更新 QComboBox"""
        try:
            rm = pyvisa.ResourceManager()
            instrument_list = rm.list_resources()
            
            # 清空現有選項
            self.ui.Scope_address.clear()
            self.ui.FG_address.clear()
            self.ui.Amp_address.clear()
            self.ui.Temp_address.clear()

            # 添加 'none' 作為預設選項
            self.ui.Scope_address.addItem("None")
            self.ui.FG_address.addItem("None")
            self.ui.Amp_address.addItem("None")
            self.ui.Temp_address.addItem("None")

            # 添加新選項到 ComboBox
            for address in instrument_list:
                self.ui.Scope_address.addItem(address)
                self.ui.FG_address.addItem(address)
                self.ui.Amp_address.addItem(address)
                self.ui.Temp_address.addItem(address)

            self.statusBar().showMessage("Instrument addresses refreshed.")
        
        except Exception as e:
            self.statusBar().showMessage(f"Failed to refresh instruments: {str(e)}")

    def connect_instruments(self):
        """
        更改連接時的各樣選擇，並進行初始設置
        """
        status_message = ""  # 用來收集所有的連接狀態訊息
        
        # 通用的儀器連接函數
        def connect_device(device_class, address, device_name):
            if address == "None" or not address:
                return f"Failed to connect to {device_name}, Please select a valid instrument address.\n"
            try:
                device = device_class(address)
                idn = device.query_identity()  # 查詢 IDN
                return f"{device_name}:{idn}.\n", device
            except Exception as e:
                return f"Failed connect to {device_name}: {str(e)}.\n", None
                
        # 連接示波器 MSO 64B
        mso_message, self.mso = connect_device(MSO_64B, self.ui.Scope_address.currentText(), "Scope")
        parts = mso_message.split(',')
        mso_connection = ','.join(parts[:2])# 保留前兩個部分        
        status_message += f"{mso_connection}.\n"
        if self.mso:
            print("MSO connected successfully.")
        else:
            self.update_status("MSO failed to connect")
    
        # 連接訊號產生器 AFG 31101
        afg_message, self.afg = connect_device(AFG_31101, self.ui.FG_address.currentText(), "Signal Generator")
        parts = afg_message.split(',')
        afg_connection = ','.join(parts[:2])# 保留前兩個部分
        status_message += f"{afg_connection}.\n"
        if self.afg:
            print("AFG connected successfully.")
        else:
            self.update_status("AFG failed to connect")
    
        # 連接功率放大器 RF150A100D
        rf_message, self.rf = connect_device(RF_150A100D, self.ui.Amp_address.currentText(), "Power Amplifier")
        parts = rf_message.split(',')
        rf_connection = ','.join(parts[:2])# 保留前兩個部分
        status_message += f"{rf_connection}.\n"
        if self.rf:
            print("RF Amplifier connected successfully.")
        else:
            self.update_status("RF Amplifier failed to connect")
    
        # 連接溫度計 HIOKI8450
        hio_message, self.hio = connect_device(HIOKI_8450, self.ui.Temp_address.currentText(), "Thermometer")
        parts = hio_message.split(',')
        hio_connection = ','.join(parts[:2])# 保留前兩個部分
        status_message += f"{hio_connection}.\n"
        if self.rf:
            print("HIOKI Thermometer connected successfully.")
        else:
            self.update_status("HIOKI Thermometer failed to connect")
            
        # 最後一次性更新 statusBar
        self.statusBar().showMessage(status_message)

    def close_connect_instruments(self):
        print("關閉儀器")
    
        # 計算增益設置值，並將功率放大器增益設為零
        gain_set = calculate_gain_set(gain=0)
        self.rf.set_gain(gain, gain_set)
    
        # 暫停0.5秒，確保增益設置完成
        time.sleep(0.5)
    
        # 關閉訊號產生器和功率放大器
        self.afg.output_off()
        self.rf.power_off()
    
        print("AFG和RF功率放大器已成功關閉")
        self.mso.close_instrument()
        self.statusBar().showMessage("Close connection with scope")
        self.afg.close_instrument()
        self.statusBar().showMessage("Close connection with signal generator")
        self.rf.close_instrument()
        self.statusBar().showMessage("Close connection with power generator")
        self.hio.close.instrument()
        self.statusBar().showMessage("Close connection with thermometer")

    """
    Start test
    """
    def add_test_measurement(self):
        """
        add measurements
        """
        self.mso.measure_mean('MEAS6','CH1')  
        self.mso.measure_mean('MEAS7','CH2')  
        self.mso.measure_mean('MEAS8','CH3')  
        self.statusBar().showMessage("Add measurment of MEAN")
        self.mso.math_add('MATH1', 'Ch1-Meas6')
        self.mso.off_display('MATH1')
        self.mso.math_add('MATH2', 'Ch2-Meas7')
        self.mso.off_display('MATH2')
        self.mso.math_add('MATH3', 'Ch3-Meas8')
        self.mso.off_display('MATH3')
        self.mso.math_add('MATH4', 'Math1/10')
        self.mso.set_math_label('MATH4','A')
        self.mso.math_add('MATH5', 'Intg(Math2)')
        self.mso.set_math_label('MATH5','T')
        self.mso.set_math_autoscale()
        self.statusBar().showMessage("Add MATHS")
        self.mso.set_acquistion('32')
        # set_cursor_range(self.afg, self.mso)
        # self.statusBar().showMessage("Set cursor range")
        self.mso.set_gating_type() #調整measurement的gating type
        self.mso.measure_delta_B() #執行測量Bmax&Bmin
        self.mso.power_function('1','MATH2', 'MATH4')
        self.mso.power_function('2','MATH3', 'MATH4')
        self.statusBar().showMessage("Add power function")
        self.mso.off_display('MATH6')
        self.mso.off_display('MATH7')
        self.mso.off_display('MATH8')
        self.mso.off_display('MATH9')
        self.mso.result_table() #新增measurement result table的介面

    def start_test(self):
        try:
            core_type = self.ui.core_type.text()
            core_OD = float(self.ui.core_od.text())
            core_ID = float(self.ui.core_id.text())
            core_He = float(self.ui.core_he.text())
            turn = float(self.ui.core_turn.text())
            frequency = float(self.ui.test_frequency_value.text())
            start_deltaB = float(self.ui.test_start_deltaB_value.text())
            end_deltaB = float(self.ui.test_end_deltaB_value.text())
            step_of_deltaB = float(self.ui.test_deltaB_step_value.text())
            file_name = self.ui.CSV_file_name.text()
            rf_state = self.rf.query_state()
            amplitude_state = rf_state.replace('STATE= ', '').strip()

            # 調用檢查參數函式
            errors = check_parameters(core_type, core_OD, core_ID, core_He, turn, frequency, start_deltaB, end_deltaB, step_of_deltaB, file_name, amplitude_state)
            if errors:
                QtWidgets.QMessageBox.warning(self, "輸入錯誤", "\n".join(errors))
                return

        except ValueError as e:
            QtWidgets.QMessageBox.warning(self, "輸入錯誤", f"請確認所有參數均為有效數值。錯誤訊息：{str(e)}")
            return
        
        self.statusBar().showMessage("Start test power loss of core")
        # 初始化時間追蹤列表
        start_time = time.time()
        measurement_times = []
        accumulated_times = []
        core_effective_length, core_effective_area, core_effective_volume = calculate_core_parameters(core_He, core_OD, core_ID)
        """
        Set logger channel
        """
        self.hio.set_channel('CH1_1', 'ON')
        self.hio.set_measurement_mode('CH1_1', 'TC')
        self.hio.set_rtd_type('CH1_1', 'K')
        """
        set Power Gain
        """
        gain = 10
        gain_set = calculate_gain_set(gain)  # 設置初始曾一直並計算增益設置值
        self.rf.set_gain(gain, gain_set)
        check_power_state(self.rf, 'ON') #判斷放大power狀態，是開或關
        """
        set Signal
        """
        shape = 'SINusoid'
        amplitude = {'magnitude': '330E-3','unit': 'VPP'}
        amplitude_limit = 0.5
        self.afg.set_waveform(shape) #設置訊號類
        frequency = float(self.ui.test_frequency_value.text())
        self.afg.set_frequency(frequency , 'kHZ') #設置訊號頻率
        self.afg.set_amplitude(amplitude['magnitude'], amplitude['unit']) #設置初始訊號振幅
        check_output_state(self.afg, 'ON') #判斷輸出狀態，是開或關
        """
        Set cursor range
        """
        set_cursor_range(self.afg, self.mso)
        """
        Set test parameter
        """
        expect_deltaB = generate_expect_deltaB(start_deltaB, end_deltaB, step_of_deltaB)
        sequences = list(range(1, len(expect_deltaB) + 1))
        deskew_vref = 1
        print(f"start deltaB:{start_deltaB}")
        print(f"end deltaB:{end_deltaB}")
        print(f"step_of_deltaB deltaB:{step_of_deltaB}")
        print(f"expect deltaB:{expect_deltaB}")
        print(f"sequences:{sequences}")

        # 啟動工作者執行測量
        self.worker = AutoTestWorker(sequences, expect_deltaB, self.mso, self.rf, self.afg, self.hio, core_effective_area, core_effective_volume, turn, gain, amplitude, amplitude_limit, deskew_vref, start_time, measurement_times, accumulated_times)
        self.worker.result_ready.connect(self.add_result_to_table)
        self.worker.test_complete.connect(
            lambda final_data: self.save_to_csv(core_type, core_effective_length, core_effective_area, core_effective_volume, turn, shape, frequency, expect_deltaB, final_data)
        )
        self.worker.start()

    def stop(self):
        """停止測試"""
        self.is_running = False

    def save_to_csv(self, core_type, core_effective_length, core_effective_area, core_effective_volume, turn, shape, frequency, expect_deltaB, final_data): 
        data = {
            ("core parameter", "Type"): [core_type] * len(final_data['sequences']),
            ("core parameter", "Le(mm)"): [core_effective_length] * len(final_data['sequences']),
            ("core parameter", "Ae(mm^2)"): [core_effective_area] * len(final_data['sequences']),
            ("core parameter", "Ve(mm^3)"): [core_effective_volume] * len(final_data['sequences']),
            ("core parameter", "Turns"): [turn] * len(final_data['sequences']),
            ("Test environment (R=10)", "Sequence"): final_data['sequences'],
            ("Test environment (R=10)", "Waveform"): [shape] * len(final_data['sequences']),
            ("Test environment (R=10)", "Frequency(kHz)"): [frequency] * len(final_data['sequences']),
            ("Test environment (R=10)", "Expect deltaB(T)"): expect_deltaB,            
            ("Test environment (R=10)", "Vin(Vpp)"): [final_data['vin_amplitude'][seq] for seq in final_data['sequences']],
            ("Test environment (R=10)", "Gain"): [final_data['power_gain'][seq] for seq in final_data['sequences']],
            ("flux density (DS=0)", "delta B(T)"): [final_data['delta_B'][seq] for seq in final_data['sequences']],
            ("DS=0", "Pcore(W)"): [v for seq, v in final_data['Pcore_true_power']['1']],
            ("DS=0", "Pair(W)"): [v for seq, v in final_data['PL_true_power']['2']],
            ("flux density (DS=1)", "delta B(T)"): [final_data['delta_B_deskew'][seq] for seq in final_data['sequences']],
            ("DS=1", "Pcore(W)"): [v for seq, v in final_data['Pcore_true_power_deskew']['1']],
            ("DS=1", "Pair(W)"): [v for seq, v in final_data['PL_true_power_deskew']['2']],
            ("Final calculate","K"):[final_data['k_factor'][seq] for seq in final_data['sequences']],
            ("Final calculate","Real Pcore(W)"):[final_data['Pcore'][seq] for seq in final_data['sequences']],
            ("Final calculate","Real Pcv(kW/m^3)"):[final_data['Pcore_cv'][seq] for seq in final_data['sequences']],
            ("Final calculate", "Temperature(degree C)"): [final_data['temperature'][seq] for seq in final_data['sequences']],
            ("Final calculate", "Spend time(sec)"): [format_time(duration) for duration in final_data['measurement_times']],
            ("Final calculate", "Accumulated time(sec)"): [format_time(accumulated_time) for accumulated_time in final_data['accumulated_times']],
        }

        # 建立MultiIndex
        index = pd.MultiIndex.from_tuples(data.keys())
    
        # 建立DataFrame
        df = pd.DataFrame(data)  # 用代號表示整個二維資料
        df.columns = index
    
        # 顯示DataFrame
        print(df)  # 列出來

        file_name = self.ui.CSV_file_name.text()
        if not file_name.strip():
            file_name = "update_your_file_name"  # 若為空字串，設置一個默認名稱
        # 儲存為 CSV 文件
        df.to_csv(f'{file_name}.csv', index=False)  # 將資料保存為 CSV 檔

    def stop_test(self):
        """停止測試"""
        if hasattr(self, 'worker') and self.worker.isRunning():
            self.worker.stop()  # 調用 AutoTestWorker 中的 stop() 方法
            self.worker.quit()  # 結束執行緒
            self.worker.wait()  # 等待執行緒完全停止


    def add_result_to_table(self, index, deltaB, pcv, temp, time):
        """
        將測量結果插入到result_table中
        :param index: 測量次數
        :param delta_B: 測量的Delta B值
        :param pcv: 計算出的Pcv值
        :param temp: 查詢的溫度
        :param time: 花費的測量時間      
        """
        # 插入新的行
        row_position = self.ui.result_table.rowCount()
        self.ui.result_table.insertRow(row_position)

        # 格式化數據至小數點後三位
        delta_B_formatted = f"{round(deltaB, 3):.3f}"
        pcv_formatted = f"{round(pcv, 3):.3f}"
        temp_formatted = f"{round(float(temp[0]), 3):.3f}" if temp else "N/A"  # 取第一個溫度數據並顯示到小數點後三位
        time_formatted = format_time(time)  # 將花費時間進行格式化
    
        # 插入測量結果
        self.ui.result_table.setItem(row_position, 0, QtWidgets.QTableWidgetItem(delta_B_formatted))
        self.ui.result_table.setItem(row_position, 1, QtWidgets.QTableWidgetItem(pcv_formatted))
        self.ui.result_table.setItem(row_position, 2, QtWidgets.QTableWidgetItem(temp_formatted))
        self.ui.result_table.setItem(row_position, 3, QtWidgets.QTableWidgetItem(time_formatted))

        # 刷新 UI 顯示
        QtWidgets.QApplication.processEvents()  # 確保每次插入後即時顯示更新

    """
    caucualte core
    """
    def calculate_core(self):
        """
        點擊按鈕後計算鐵芯參數
        """
        try:
            # 從 UI 抓取數值
            core_He = self.ui.core_he.value()
            core_OD = self.ui.core_od.value()
            core_ID = self.ui.core_id.value()

            # 計算鐵芯的長度、截面積和體積
            core_effective_length, core_effective_area, core_effective_volume = calculate_core_parameters(core_He, core_OD, core_ID)

            # 更新到 UI 上顯示結果
            self.ui.core_length.setText(f"{core_effective_length:.5f}")
            self.ui.core_area.setText(f"{core_effective_area:.5f}")
            self.ui.core_volume.setText(f"{core_effective_volume:.5f}")

        except ValueError:
            # 若計算過程有錯，顯示錯誤訊息
            QtWidgets.QMessageBox.warning(self, "計算錯誤", "計算鐵芯參數時出現錯誤，請檢查輸入值。")

    """
    Send the status message
    """
    def update_status(self, message):
        # 更新狀態欄顯示訊息
        self.statusBar().showMessage(message)

In [ ]:
def main():
    app = QtWidgets.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

if __name__ == "__main__":
    main()

Connected to MSO64B.
TEKTRONIX,MSO64B,C048992,CF:91.1CT FV:1.44.3.433

MSO connected successfully.
Connected to AFG_31101.
TEKTRONIX,AFG31101,C016504,SCPI:99.0 FV:1.6.1

AFG connected successfully.
Connected to RF150A100D.
AR-RF/MICROWAVE-INST,150A100D,1.0

RF Amplifier connected successfully.
Connected to HIOKI8450.
HIOKI,LR8450,220139566,V2.21

HIOKI Thermometer connected successfully.
add measurent MEAS6 of CH1
add measurent MEAS7 of CH2
add measurent MEAS8 of CH3
Add MATH1 to calculate Ch1-Meas6
turn MATH1 display off
Add MATH2 to calculate Ch2-Meas7
turn MATH2 display off
Add MATH3 to calculate Ch3-Meas8
turn MATH3 display off
Add MATH4 to calculate Math1/10
set label of MATH4 be A 
Add MATH5 to calculate Intg(Math2)
set label of MATH5 be T 
enables the autoscaling the math4 in the specified Waveform View
enables the autoscaling the math5 in the specified Waveform View
set acquisition in average mode and waveform acquire number be 32
take measurements on the portion of the wavefor